### 라이브러리 업로드

In [1]:
import time
import hashlib
from queue import Queue
import heapq

### 블록 클래스 생성

In [2]:
class Block:
    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index # 블록 인덱스
        self.previous_hash = previous_hash # 이전 해시 저장
        self.timestamp = timestamp # 타임스탬프
        self.data = data # 블록 내에 들어가는 데이터
        self.hash = hash # 현재 해시 저장

### 블록체인 클래스

In [3]:
class Blockchain:
    def __init__(self):
        self.chain = [] # 블록을 연결할 단순 연결 리스트(이는 해시로 연결됨)
        self.users = {}  # 유저를 맵 형태로 저장 (key : UserID, value : User data)
        self.tokens = {} # 유저를 맵 형태로 저장 (key : TokenID, value : Token data)
        
    # 제네시스 블록을 생성하는 메서드
    def create_genesis_block(self):
        genesis_block = Block(0, "0", int(time.time()), "Genesis Block", self.hash_block("0")) # 제네시스 블록 생성
        self.chain.append(genesis_block) # 제네시스 블록을 블록체인에 추가

    # transaction(유저 등록, 토큰 발행, 매매, 배당)이 일어나면 블록에 추가하는 메서드
    def add_block(self, transactions):
        # 데이터 생성
        previous_block = self.chain[-1]
        index = previous_block.index + 1
        timestamp = int(time.time())
        previous_hash = previous_block.hash
        data = str(transactions)

        # 블록체인에 추가
        hash_value = self.hash_block(previous_hash + str(index) + str(timestamp) + data)
        new_block = Block(index, previous_hash, timestamp, data, hash_value)
        self.chain.append(new_block)

    # 해시 변환하는 매서드
    def hash_block(self, data): 
        return hashlib.sha256(data.encode()).hexdigest()
        
    # 블록체인 내의 전체 내용을 출력하는 함수
    def print_blocks(self): 
        for block in self.chain:
            print(f"Index: {block.index}")
            print(f"Previous Hash: {block.previous_hash}")
            print(f"Timestamp: {block.timestamp}")
            print(f"Data: {block.data}")
            print(f"Hash: {block.hash}")
            print("------------------------")    
    # 블록체인 내에 발행된 토큰 정보를 출력하는 함수
    def print_tokens(self):
        for token_id, token_info in self.tokens.items():
            print(f"토큰 ID: {token_id}")
            print(token_info)
            print("------------------------")

    # Token ID를 입력하여 개별 토큰 정보를 출력하는 함수
    def search_token(self, token_id): 
        return self.tokens.get(token_id, None)

### 토큰 클래스

In [4]:
class Token:
    def __init__(self, token_id, name, price, issuer, underlying_asset, underlying_price, underlying_location):
        self.token_id = token_id # 토큰 아이디
        self.name = name # 토큰명
        self.price = price # 초기 가격
        self.issuer = issuer # 발행자
        self.underlying_asset = underlying_asset # 기초자산 정보
        self.underlying_price = underlying_price # 기초자산 가격
        self.underlying_location = underlying_location # 기초자산 위치

    # 맵 자료구조 변환 메서드
    def to_dict(self):
        return {
            "token_id": self.token_id,
            "name": self.name,
            "price": self.price,
            "issuer": self.issuer,
            "underlying_asset": self.underlying_asset,
            "underlying_price": self.underlying_price,
            "underlying_location": self.underlying_location
        }

    # 토큰 발행 메서드
    def token_issue_to_blockchain(self, blockchain):
        token_data = self.to_dict() # 맵 자료구조로 변환
        blockchain.tokens[self.token_id] = token_data # Key를 TokenID로 설정
        blockchain.add_block(token_data) # 블록체인에 추가

### 유저 클래스

In [5]:
# 유저 클래스
class User:
    def __init__(self, user_id, name, account_address, account_balance, num_tokens_held, blockchain):
        self.user_id = user_id # 유저 아이디
        self.name = name # 유저명
        self.account_address = account_address # 계좌 주소
        self.account_balance = account_balance # 계좌 잔고
        self.num_tokens_held = num_tokens_held # 보유 토큰 개수
        self.blockchain = blockchain

    # 맵 자료구조로 변환 메서드
    def to_dict(self):
        return {
            "user_id": self.user_id,
            "name": self.name,
            "account_address": self.account_address,
            "account_balance": self.account_balance,
            "num_tokens_held": self.num_tokens_held
        }

    # 유저 등록 메서드
    def user_issue_to_blockchain(self, blockchain):
        user_data = self.to_dict() # 맵 자료구조로 변환
        blockchain.users[self.user_id] = user_data # Key를 UserID를 설정
        blockchain.add_block(user_data)  # 블록체인에 추가

    # 유저 정보 출력 메서드
    def print_user(self):
        print(f"User ID: {self.user_id}")
        print(f"User Name: {self.name}")
        print(f"Account Address: {self.account_address}")
        print(f"Account Balance: {self.account_balance}")
        print(f"Number of Tokens Held: {self.num_tokens_held}")
        print("------------------------")
    # 배당금 전송 메서드
    def receive_dividend(self, amount):
        self.account_balance += amount
        print(f"Received dividend: {amount} won. New account balance: {self.account_balance} won.")

### 주문 클래스

In [6]:
class Order:
    def __init__(self, user, token_id, category, price, quantity):
        self.user = user # 주문자명
        self.token_id = token_id # 주문 토큰
        self.category = category # 매수 or 매도
        self.price = price # 주문 가격
        self.quantity = quantity # 주문량
    
    # 가격 비교 메서드
    def __lt__(self, other):
        return self.price < other.price

    def __eq__(self, other):
        return self.price == other.price

### 매매 클래스

In [16]:
class Exchange:
    def __init__(self, blockchain):
        self.orders = {}  # 각 토큰별 주문 저장 딕셔너리
        self.completed_transactions = Queue()  # 완료된 거래 저장 큐
        self.blockchain = blockchain  # 블록체인 객체
        self.buy_orders_heap = []  # 매수 주문을 위한 최대 힙
        self.sell_orders_heap = []  # 매도 주문을 위한 최소 힙
        
    # 주문 유형에 따라 계정 잔액 또는 토큰 수량 갱신
    def place_order(self, order, token_id):
        if order.category == 'buy':
            order.user.account_balance -= order.price * order.quantity  # 매수 주문 시 잔액 감소
        elif order.category == 'sell':
            # 매도 주문 시 토큰 수량 감소
            if token_id in order.user.num_tokens_held:
                order.user.num_tokens_held[token_id] -= order.quantity
            else:
                print("Error: Not enough tokens to sell")
                return
        # 매수 주문을 최대힙, 매도 주문을 최소 힙에 추가
        if order.category == 'buy':
            heapq.heappush(self.buy_orders_heap, (-order.price, order))
        elif order.category == 'sell':
            heapq.heappush(self.sell_orders_heap, (order.price, order))

        self.match_orders(token_id)

    
    # 매수 및 매도 주문을 생성하고 처리하는 함수
    def create_and_place_orders(self, user, token, category, price, quantity):
        order = Order(user, token.token_id, category, price, quantity)
        self.place_order(order, token.token_id)  

    # 주문 일치 처리하는 함수
    def match_orders(self, token):
        while self.buy_orders_heap and self.sell_orders_heap:
            if -self.buy_orders_heap[0][0] >= self.sell_orders_heap[0][0]:
                buy_order = heapq.heappop(self.buy_orders_heap)[1]
                sell_order = heapq.heappop(self.sell_orders_heap)[1]
                quantity = min(buy_order.quantity, sell_order.quantity)

                # 체결된 거래 처리
                completed_trade = self.execute_trade(buy_order, sell_order, quantity)
                if completed_trade:
                    self.completed_transactions.put(completed_trade)


    def execute_trade(self, buy_order, sell_order, quantity):
        # 체결된 거래 정보 생성 및 저장
        completed_trade = {
            'buy_order': [{'user': buy_order.user.user_id},{'token_id':buy_order.token_id},{'category':buy_order.category},
                         {'price': buy_order.price}, {'quantity':buy_order.quantity}], 
            'sell_order': [{'user': sell_order.user.user_id},{'token_id':sell_order.token_id},{'category':sell_order.category},
                         {'price': sell_order.price}, {'quantity':sell_order.quantity}],
            'price': sell_order.price, 
            'quantity': quantity
        }

        # 매수 주문을 한 사용자의 토큰 수량 증가
        if buy_order.category == 'buy':
            if buy_order.token_id in buy_order.user.num_tokens_held:
                buy_order.user.num_tokens_held[buy_order.token_id] += quantity
            else:
                buy_order.user.num_tokens_held[buy_order.token_id] = quantity

        # 매도 주문을 한 사용자의 계좌에서 체결된 금액만큼 감소
        if sell_order.category == 'sell':
            sell_order.user.account_balance += sell_order.price * quantity

        
        return completed_trade
    # 최저 매도 주문 가격 반환
    def get_lowest_sell_order_price(self):
        return self.sell_orders_heap[0][0] if self.sell_orders_heap else None
        
    # 최고 매수 주문 가격 반환
    def get_highest_buy_order_price(self):
        return -self.buy_orders_heap[0][0] if self.buy_orders_heap else None
    
    # 완료된 거래 목록 반환
    def create_block(self):
        transactions = []
        while not self.completed_transactions.empty() and len(transactions) < 10:
            transactions.append(self.completed_transactions.get())
        if transactions:
            self.blockchain.add_block(transactions)
    
    def get_buy_orders(self):
        """매수 주문 목록을 반환합니다."""
        return [(order[1].user.user_id, -order[0], order[1].quantity) for order in self.buy_orders_heap]

    def get_sell_orders(self):
        """매도 주문 목록을 반환합니다."""
        return [(order[1].user.user_id, order[0], order[1].quantity) for order in self.sell_orders_heap]
    
    def get_completed_transactions(self):
        """체결된 거래 목록을 반환합니다."""
        completed_transactions_list = []
        while not self.completed_transactions.empty():
            transaction = self.completed_transactions.get()
            completed_transactions_list.append(transaction)
            self.completed_transactions.put(transaction)  # 거래 정보를 다시 큐에 추가
        return completed_transactions_list
    
    def update_balance_and_tokens(self, token_id, token_amount, balance_change):
        """계좌 잔액과 토큰 수량을 업데이트합니다."""
        self.account_balance += balance_change
        self.num_tokens_held = self.num_tokens_held + token_amount

In [17]:
# 토큰당 배당금을 저장하는 맵 자료구조
token_price_map = {}

class Dividend:
    @staticmethod
    # 배당금 계산을 위해 토큰 보유량을 추출
    def calculate_dividend(num_tokens_held):
        sum = 0
        tokens = list(num_tokens_held.keys())
        print(tokens)
        print(num_tokens_held[tokens[0]])
        
        # 각 토큰에 대한 배당금 계산 (토큰 수량 * 토큰 가격)
        for i in range(len(tokens)):
            
            token_price = token_price_map[tokens[i]]
            quantity = num_tokens_held[tokens[i]]
            
            sum += quantity * token_price
            
        return sum

    @staticmethod
    def issue_to_blockchain(blockchain, user):
    
        # 사용자가 보유한 토큰에 대한 배당금 계산
        dividend_price = Dividend.calculate_dividend(user.num_tokens_held)
        
        # 배당금 정보를 거래 목록에 추가
        transactions = {"dividend": dividend_price}
        blockchain.add_block(transactions)
        # 계산된 배당금
        dividend_amount = dividend_price

        # 계좌에 배당금 전송
        user.receive_dividend(dividend_amount)
        
        # 사용자 정보를 블록체인에 업데이트
        user.user_issue_to_blockchain(blockchain)

### 실행 예시

#### 1. 제네시스 블록 생성 / 토큰 발행 / 유저 등록

In [18]:
# 블록체인 객체 및 genesis block 생성
blockchain = Blockchain()
blockchain.create_genesis_block()

# 토큰 정보 입력
token_id = "T001" #토큰 ID
token_name = "쏭토큰" #토큰 이름
token_price = 10 #토큰 가격
token_issuer = "박성우"  # 발행자명
underlying_asset = "쏭건물"  # 기초자산명
underlying_price = 20  # 기초자산 가격
underlying_location = "아주대학교 일신관"  # 기초자산위치

# Song_token라는 토큰 객체 생성 
Song_token = Token(token_id, token_name, token_price, token_issuer, 
                   underlying_asset, underlying_price, underlying_location)

# Song_token 발행
Song_token.token_issue_to_blockchain(blockchain)

# 토큰 정보 입력
token_id = "T002" #토큰 ID
token_name = "히지토큰" #토큰 이름
token_price = 15 #토큰 가격
token_issuer = "정희지"  # 발행자명
underlying_asset = "히지건물"  # 기초자산명
underlying_price = 25  # 기초자산 가격
underlying_location = "아주대 다산관"  # 기초자산위치

# Heeji_token라는 토큰 객체 생성
Heeji_token = Token(token_id, token_name, token_price, token_issuer, underlying_asset,
                    underlying_price, underlying_location)

# Heeji_token 발행
Heeji_token.token_issue_to_blockchain(blockchain)


# 유저 정보 입력
user_id = "U001" # 유저 ID
user_name = "박성우" # 유저 이름 
account_address = "0x123456789" # 유저 주소
account_balance = 10000 # 유저 계좌 잔고
num_tokens_held = {'T001': 20} # 유저 보유 토큰 and 토큰 수량

# Song_user라는 유저 객체 생성
Song_user = User(user_id, user_name, account_address, account_balance, num_tokens_held, blockchain = blockchain)

# Song_user 유저 등록
Song_user.user_issue_to_blockchain(blockchain)

# 유저 정보 입력
user_id = "U002" # 유저 ID
user_name = "임진영" # 유저 이름
account_address = "0x123456798" # 유저 주소
account_balance = 15000 # 유저 계좌 잔고
num_tokens_held = {'T001': 12, 'T002' : 10} # 유저 보유 토큰 and 토큰 수량

# Young_user라는 유저 객체 생성
Young_user = User(user_id, user_name, account_address, account_balance, num_tokens_held, blockchain = blockchain)

# Young_user 유저 등록
Young_user.user_issue_to_blockchain(blockchain)

# 블록체인 정보 출력
blockchain.print_blocks()

Index: 0
Previous Hash: 0
Timestamp: 1702109950
Data: Genesis Block
Hash: 5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9
------------------------
Index: 1
Previous Hash: 5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9
Timestamp: 1702109950
Data: {'token_id': 'T001', 'name': '쏭토큰', 'price': 10, 'issuer': '박성우', 'underlying_asset': '쏭건물', 'underlying_price': 20, 'underlying_location': '아주대학교 일신관'}
Hash: 404f4f029cff1877b5bea94456eeb708610acb50d4beb1ce652c6c303156c247
------------------------
Index: 2
Previous Hash: 404f4f029cff1877b5bea94456eeb708610acb50d4beb1ce652c6c303156c247
Timestamp: 1702109950
Data: {'token_id': 'T002', 'name': '히지토큰', 'price': 15, 'issuer': '정희지', 'underlying_asset': '히지건물', 'underlying_price': 25, 'underlying_location': '아주대 다산관'}
Hash: 57c6492aa30e724e094525e9cb65facd4a99cd8c44fe4479c41cf1274f6cc621
------------------------
Index: 3
Previous Hash: 57c6492aa30e724e094525e9cb65facd4a99cd8c44fe4479c41cf1274f6cc621
Timestamp: 1702

#### 2. 토큰 매매

In [19]:
# 거래소 초기화
exchange = Exchange(blockchain)

# 계좌 및 토큰 확인 가능
Young_user.print_user()

Song_user.print_user()

# 매수 및 매도 주문 생성 및 처리
for i in range(10):
    exchange.create_and_place_orders(Young_user, Song_token, 'buy', 100, 2)  # 100원에 매수 주문 20개
    exchange.create_and_place_orders(Young_user, Song_token, 'buy', 50, 2) # 50원에 매수 주문 20개
    
for i in range(10):
    exchange.create_and_place_orders(Song_user, Song_token, 'sell', 70, 2) # 70원에 매도 주문 20개


# 10개의 체결 거래 블록체인에 블록 생성
exchange.create_block()

# 10개 거래 블록 생성 확인
blockchain.print_blocks()

# 계좌 및 토큰 변경 확인 
Young_user.print_user()

Song_user.print_user()

User ID: U002
User Name: 임진영
Account Address: 0x123456798
Account Balance: 15000
Number of Tokens Held: {'T001': 12, 'T002': 10}
------------------------
User ID: U001
User Name: 박성우
Account Address: 0x123456789
Account Balance: 10000
Number of Tokens Held: {'T001': 20}
------------------------
Index: 0
Previous Hash: 0
Timestamp: 1702109950
Data: Genesis Block
Hash: 5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9
------------------------
Index: 1
Previous Hash: 5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9
Timestamp: 1702109950
Data: {'token_id': 'T001', 'name': '쏭토큰', 'price': 10, 'issuer': '박성우', 'underlying_asset': '쏭건물', 'underlying_price': 20, 'underlying_location': '아주대학교 일신관'}
Hash: 404f4f029cff1877b5bea94456eeb708610acb50d4beb1ce652c6c303156c247
------------------------
Index: 2
Previous Hash: 404f4f029cff1877b5bea94456eeb708610acb50d4beb1ce652c6c303156c247
Timestamp: 1702109950
Data: {'token_id': 'T002', 'name': '히지토큰', 'price': 15, 'issuer

#### 3. 배당 지급

In [11]:
# 베딩
# 토큰 별 배당 금액 할당
token_price_map['T001'] = 100
token_price_map['T002'] = 150

Dividend.issue_to_blockchain(blockchain, Song_user)
Dividend.issue_to_blockchain(blockchain, Young_user)

blockchain.print_blocks()

['T001']
0
Received dividend: 0 won. New account balance: 11400 won.
['T001', 'T002']
32
Received dividend: 4700 won. New account balance: 16700 won.
Index: 0
Previous Hash: 0
Timestamp: 1702109489
Data: Genesis Block
Hash: 5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9
------------------------
Index: 1
Previous Hash: 5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9
Timestamp: 1702109489
Data: {'token_id': 'T001', 'name': '쏭토큰', 'price': 10, 'issuer': '박성우', 'underlying_asset': '쏭건물', 'underlying_price': 20, 'underlying_location': '아주대학교 일신관'}
Hash: b1736f8d597dce79e6dae1383b30c23b44dcfe9f7f706c321b2143089d227dbb
------------------------
Index: 2
Previous Hash: b1736f8d597dce79e6dae1383b30c23b44dcfe9f7f706c321b2143089d227dbb
Timestamp: 1702109489
Data: {'token_id': 'T002', 'name': '히지토큰', 'price': 15, 'issuer': '정희지', 'underlying_asset': '히지건물', 'underlying_price': 25, 'underlying_location': '아주대 다산관'}
Hash: 6bdee37bc96b95ac9ffb8244b43987e8f81123e83fcf